# 3D Mesh Processing, Sampling Analysis, and Reconstruction

## 1. Introduction

The reconstruction of 3D structures from 2D images—Structure from Motion (SfM)—is a fundamental process in modern computer vision. The accuracy of the generated 3D model is heavily dependent on camera intrinsics, trajectory estimation, and visual overlap.

This notebook documents a quantitative analysis of a 3D surface reconstruction pipeline. The project evaluates how degrading the input image dataset (from full coverage to partial subsets) impacts the final geometric completeness and registration accuracy against a ground truth model.

The process involves:
1.  **Monocular Camera Calibration**: Determining intrinsic parameters to remove geometric distortion.
2.  **Trajectory Analysis**: Comparing estimated camera poses from the reconstruction pipeline against ground truth data.
3.  **Surface Registration (ICP)**: Aligning the reconstructed 3D meshes with a high-density reference mesh using the Iterative Closest Point algorithm to evaluate reconstruction fidelity.

---


## 2. Pre-Processing Technical Analysis

### Technical Analysis 1: Lens Distortion Modeling

When initializing the camera pipeline, correcting physical lens distortion is critical. The primary distortion model utilized is the Brown-Conrady model, which accounts for both radial and tangential aberrations.

*   **Radial Distortion**: Corrects for physical lens curvature causing barrel or pincushion effects (modeled by parameters $k_1, k_2, k_3$). Ensures straight lines in the world remain straight in the projection.
*   **Tangential Distortion**: Corrects for imperfect alignment between the lens assembly and the imaging sensor (modeled by $p_1, p_2$).
*   **Mathematical Correction**: Ideal unobservable coordinates $(x,y)$ are displaced to distorted coordinates $(x_{dist}, y_{dist})$:
    *   $x_{dist} = x(1 + k_1 r^2 + k_2 r^4 + k_3 r^6) + [2p_1 xy + p_2(r^2 + 2x^2)]$
    *   $y_{dist} = y(1 + k_1 r^2 + k_2 r^4 + k_3 r^6) + [p_1(r^2 + 2y^2) + 2p_2 xy]$

### Technical Analysis 2: Texture Coordinate Export Formats

For general-purpose texturing of the final geometry, the pipeline exports PNG or JPG files instead of heavy floating-point formats like TIFF or EXR.
*   Standard 8-bit Albedo textures are sufficient for standard physically-based rendering workflows.
*   EXR/TIFF are vital for preserving HDR linear workflow data during intermediate lighting computations but cause unnecessary bloat when deploying the final asset. 

### Technical Analysis 3: Mesh Geometry Representation

The final geometry is explicitly exported into the **STL (Stereolithography)** format.
*   **Justification**: STL encodes raw unstructured triangulated surfaces, stripping all color, texture mappings, and scene graphs. It is the absolute industry standard for geometric validation, finite element analysis, and 3D printing. It enforces a focus purely on spatial structure and manifold integrity.

---


## 3. Monocular Camera Calibration & Intrinsics Analysis

### Methodology
Calibration extracts metric information from 2D arrays. Utilizing a checkerboard pattern with known world-space geometry, the system detects corner features across multiple views. The optimization solves for the camera's **intrinsic matrix ($K$)** and **distortion coefficients**. Minimizing the **reprojection error** ensures mathematical alignment between projected 3D coordinates and detected 2D pixels.


In [ ]:
import os
import sys

# Add src module to path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from src import mesh_metrics

print("Modules loaded successfully.")



### Technical Analysis 4: Optimization Constraints

During the calibration optimization routine, higher-order radial distortion coefficients ($k_3, k_4, k_5, k_6$) are intentionally disabled.
*   **Engineering Rationale**: For standard consumer lenses operating at moderate fields of view, 3rd-order and higher distortions are negligible. Including them expands the parameter space and often leads to **overfitting**, where the optimizer attempts to model sensor noise as geometric distortion, leading to numerical instability at image boundaries.

### Technical Analysis 5: Intrinsics Comparison

The robust chessboard calibration yields a precise intrinsics matrix with a reprojection error of ~0.20 px. 

Applying these intrinsics directly to the automatically estimated poses from the SfM backend reveals significant deviation:
*   **Full Dataset**: ~12.38 px error
*   **Half Dataset**: ~21.75 px error
*   **Quarter Dataset**: ~71.10 px error

**Observation**: Automatic self-calibration via bundle adjustment struggles to estimate pure intrinsics. It jointly optimizes 3D structure and poses against thousands of noisy features. As the dataset sparsity increases (Quarter Dataset), the backend's ability to lock the intrinsic parameters degrades catastrophically.

---


## 4. Iterative Closest Point (ICP) Surface Registration

### Methodology
Rigorously assessing reconstruction quality requires aligning the generated mesh to the exact coordinate space of a ground-truth model.
1.  **Uniform Surface Sampling**: Extracting equidistant points across the geometry to normalize density.
2.  **Global Initialization (RANSAC)**: Computing Fast Point Feature Histograms (FPFH) to bridge large translational offsets and find a deterministic rough alignment.
3.  **Local Refinement (ICP)**: Iteratively minimizing the **Point-to-Plane** distance metric between the sparse clouds to achieve sub-millimeter registration.

### Technical Analysis 6: Efficiency vs. Accuracy via Sampling

ICP calculates nearest-neighbor correspondences, an $O(N^2)$ operation heavily bottlenecked by dense geometries.
*   Extracting raw mesh vertices directly is biased. High-detail regions contain dense vertex clusters, forcing the optimizer to heavily weight curved areas over flat planes.
*   **Uniform sampling** ensures computational efficiency while guaranteeing the entire macroscopic surface contributes equally to the cost function error minimization.

### Technical Analysis 7: Dual-Metric Assessment

A single metric cannot capture registration quality. The pipeline outputs two interdependent metrics:
*   **Fitness Score**: A structural *Completeness* metric representing the percentage of source points that successfully align with target points within the distance threshold.
*   **Inlier RMSE**: A spatial *Accuracy* metric capturing the Root Mean Square Error specifically of the successfully matched patches.

Why both are required: A tiny isolated fragment of a mesh could match the ground truth perfectly (yielding near-zero RMSE) but represent only 2% of the object (terrible Fitness). Both are required to validate the reconstruction.

### Technical Analysis 8: Dataset Degradation Impact
*   **Full Dataset**: Achieves near-perfect reconstruction (96.5% Fitness, 0.003 RMSE).
*   **Partial Datasets**: Show catastrophic drops in Fitness (47-70%). The SfM pipeline starves for overlap, breaking crucial loop closures. 
*   **Conclusion**: Dataset sparsity primarily attacks the global completeness of the geometry (leaving massive holes) rather than the localized coordinate accuracy of the surviving structural fragments.

---


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Visualizing results
img_dir = os.path.join("..", "report", "result_images")

fig, ax = plt.subplots(2, 1, figsize=(15, 12))
try:
    ax[0].imshow(mpimg.imread(os.path.join(img_dir, "sampling_comparison.png")))
    ax[0].axis('off')
    ax[0].set_title("Surface Sampling Analysis")
    
    ax[1].imshow(mpimg.imread(os.path.join(img_dir, "metric_analysis.png")))
    ax[1].axis('off')
    ax[1].set_title("Registration Metrics")
    plt.tight_layout()
    plt.show()
except FileNotFoundError:
    print("Result images not found. Please run the generation script first.")

